# HySwash: A hybrid method for nearshore wave processes

![sketch](./assets/hyswash-sketch.png)

If you want to run this notebook using the pickle files exported in Part 1, execute the code below. Otherwise, you can skip it.

In [ ]:
import requests
import tarfile
import os

url = "https://geoocean.sci.unican.es/data/HyVeggy_exported.tar.gz"
response = requests.get(url, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|gz")
file.extractall(path=os.getcwd())

ReadError: not a gzip file

In [6]:
import os
import os.path as op
import numpy as np
import utils.plotting
import bluemath_tk.topo_bathy.profiles
from bluemath_tk.datamining.lhs import LHS
from bluemath_tk.datamining.mda import MDA
from bluemath_tk.waves.series import waves_dispersion 
from bluemath_tk.wrappers.swash.swash_wrapper import HySwashVeggyModelWrapper
from bluemath_tk.core.io import load_model
from bluemath_tk.core.plotting.scatter import plot_scatters_in_triangle
import xarray as xr
import matplotlib.pyplot as plt
from utils.plotting import animate_case_propagation
from IPython.display import HTML
import utils.ChySwash
import pandas as pd

root_dir = os.getcwd()
#output_dir = "/lustre/geocean/DATA/hidronas1/valva/Veggy"
output_dir = os.path.join(root_dir, "output")
templates_dir = os.path.join(root_dir, "templates", "HyVeggy")
export_dir = op.join(root_dir, "HyVeggy_exported")

Load swash_model and mda from a pickle file and postprocessed_output NetCDF file

In [52]:
mda=load_model(op.join(export_dir, "mda_model.pkl"))
swash_model=load_model(op.join(export_dir, "swash_model.pkl"))
depth_file=op.join(templates_dir,"depth.bot")
depth_array = np.loadtxt(depth_file)
postprocessed_output = xr.open_dataset(op.join(export_dir, "output_postprocessed_clean.nc"))

In [133]:
rbf_Ru=load_model(op.join(export_dir, "rbf_Ru2_model.pkl"))
data=mda.centroids.iloc[378:500].copy()
Rusp=rbf_Ru.predict(mda.centroids.iloc[378:500])
data["Rusp"]=Rusp.Ru2.values
data
RusReal=postprocessed_output.Ru2.values[378:500]
data["RusReal"]=RusReal
max_Rus_real=np.max(RusReal)
min_Rus_real=np.min(RusReal)

data["Error_por_ciento"]=np.abs(data["Rusp"]-data["RusReal"])*100/max_Rus_real
data

,Hs,Hs_L0,Wv,hv,Nv,Rusp,RusReal,Error_por_ciento
378,2.477757,0.003517,199.970506,0.796264,648.477391,1.243519,1.514656,13.654463
379,2.368824,0.003601,63.886927,0.726844,150.846555,1.514655,0.806936,35.640796
380,1.429801,0.003378,15.038895,0.707869,344.093247,0.806935,0.401208,20.432459
381,0.976242,0.005761,191.314410,0.471255,993.823872,0.401208,1.151054,37.762281
382,2.252546,0.006186,12.948584,0.652941,952.906291,1.151054,0.616248,26.932875
...,...,...,...,...,...,...,...,...
495,2.222633,0.005147,60.923325,0.588960,536.053030,1.199585,0.898120,15.181778
496,2.897771,0.003181,174.494775,0.402380,771.081497,1.811862,0.481816,66.981214
497,1.533502,0.003454,12.287627,1.073425,624.594570,0.898120,0.985512,4.401093
498,1.159829,0.004233,130.266947,1.418113,556.237460,0.481816,1.066520,29.445731


## 5. Reconstruction: Principal Component Analysis (PCA) & Radial Basis Fucntions (RBF)

The reconstruction of the time series of wave parameters in the position of the buoy is carried out by an interpolation technique based on radial basis functions (RBF), a scheme wich is very convenient for scatter and multivariate data. The RBF approximation has been applied successfully in many fields, usually with better results than other interpolation methods (Hardy, 1990).
    
Suppose that $f=f(x)$ is the real-valued function that we want to approximate. We are given M scattered data points $\{x_1,..., x_M\}$ of dimension $\textit{n}$ and the associated real function values $\{f_1, ..., f_M\}$, being $f_i = f(x_j), j = 1,...,M$. The RBF interpolation method consists of a weighted sum of radially symmetric basic functions located at the data points. The approximation function is assumed to be of the form:
$$RBF(x) = p(x) + \sum\limits_{j=1}^M a_j\Phi{\large (}{\large \parallel}{x - x_j}{\large \parallel}{\large )}$$

### Hs reconstruction

In [ ]:
from bluemath_tk.datamining.pca import PCA
from bluemath_tk.interpolation.rbf import RBF

postprocessed_output=postprocessed_output.copy(deep=True)

# Apply PCA to the postprocessed output
pca = PCA()
_pcs_ds = pca.fit_transform(
    data=postprocessed_output,
    vars_to_stack=["Hs"],
    coords_to_stack=["Xp"],
    pca_dim_for_rows="case_num",
    value_to_replace_nans={"Hs": 0.0},
)

# Apply RBF reconstruction
rbf = RBF()
rbf.fit(
    subset_data=mda.centroids.iloc[postprocessed_output["case_num"].values, :],
    target_data=pca.pcs_df,
)

pca.save_model(
    model_path=op.join(export_dir, "pca_model.pkl"),
)
rbf.save_model(
    model_path=op.join(export_dir, "rbf_model.pkl"),
)

In [3]:
pca=load_model(op.join(export_dir, "pca_model.pkl"))
rbf=load_model(op.join(export_dir, "rbf_model.pkl"))

Visualize the effect of the plants on Hs

In [41]:
import importlib
import utils.plotting
importlib.reload(utils.plotting)
from utils.plotting import show_graph_for_different_parameters

variables_to_analyse_in_metamodel = ["Hs", "Hs_L0", "Wv","hv","Nv"]
lhs_parameters = {
    "num_dimensions": 5,
    "num_samples": 11000,
    "dimensions_names": variables_to_analyse_in_metamodel,
    "lower_bounds": [0.5, 0.003, 0, 0, 0],
    "upper_bounds": [3, 0.01, 200, 1.5, 1000],
}
utils.plotting.show_graph_for_different_parameters(pca=pca, rbf=rbf,lhs_parameters=lhs_parameters,depthfile=depth_file)

NameError: name 'pca' is not defined

In [106]:
target_data=postprocessed_output.Ru2.values,
target_data

(array([0.64255 , 0.274696, 0.371972, 1.998982, 0.681592, 1.327704,
        0.55266 , 0.383524, 2.15714 , 0.420728, 1.058616, 1.2333  ,
        0.393332, 1.672244, 2.087584, 1.476364, 0.468964, 0.334178,
        0.285992, 1.30995 , 0.542772, 1.748508, 0.891874, 0.42702 ,
        1.33858 , 0.42981 , 1.87348 , 0.633192, 0.906892, 0.246856,
        0.5394  , 0.395558, 0.860192, 1.160892, 1.414204, 0.3273  ,
        0.76324 , 2.16496 , 1.49368 , 0.44646 , 0.404432, 1.62222 ,
        0.5428  , 0.879324, 1.231402, 1.105852, 0.26104 , 0.628224,
        1.666016, 0.5217  , 0.732438, 0.238544, 0.516916, 1.573272,
        1.1808  , 2.24236 , 0.869272, 0.977184, 1.07862 , 1.551666,
        0.807292, 1.388852, 1.139718, 0.67022 , 0.801062, 1.1999  ,
        1.183652, 0.312872, 0.29406 , 1.106486, 1.155792, 1.156692,
        1.66996 , 1.076122, 0.546924, 1.5879  , 0.4727  , 0.324512,
        0.31583 , 0.380658, 0.404658, 0.314856, 1.661696, 1.07861 ,
        0.7673  , 0.46143 , 0.582602, 0.765732, 

In [131]:
target_data=postprocessed_output["Ru2"]



# Convert target_data xarra.dataarray to pandas dataframe
target_data_df = target_data.to_dataframe()

centroids=mda.centroids.iloc[postprocessed_output["case_num"].values, :].reset_index(drop=True)

# Create a pandas dataframe

Ru=pd.DataFrame(postprocessed_output["Ru2"].values, columns=["Ru2"])
rbf_Ru=RBF()
rbf_Ru.fit(
    subset_data=centroids,
    target_data=Ru,
)
rbf_Ru.save_model(
    model_path=op.join(export_dir, "rbf_Ru2_model.pkl"),
)




        ---------------------------------------------------------------------------------
        | Initializing RBF interpolation model with the following parameters:
        |    - sigma_min: 0.001
        |    - sigma_max: 0.1
        |    - sigma_diff: 0.0001
        |    - sigma_opt: None
        |    - kernel: gaussian
        |    - smooth: 1e-05
        | For more information, please refer to the documentation.
        | Recommended lecture: https://link.springer.com/article/10.1023/A:1018975909870
        ---------------------------------------------------------------------------------
        


In [59]:
rbf_Ru=load_model(op.join(export_dir, "rbf_model.pkl"))

In [39]:
Hs=2.5
Hs_Lo=0.005


rbf_Ru=load_model(op.join(export_dir, "rbf_Ru_model.pkl"))
variables_to_analyse_in_metamodel = ["Wv","hv","Nv"]
lhs_parameters = {
    "num_dimensions": 3,
    "num_samples": 500,
    "dimensions_names": variables_to_analyse_in_metamodel,
    "lower_bounds": [0, 1.4, 999],
    "upper_bounds": [200, 1.5, 1000],
}

lhs = LHS(
    num_dimensions=lhs_parameters.get("num_dimensions"),
)

lhs_dataset = lhs.generate(
    dimensions_names=lhs_parameters.get("dimensions_names"),
    lower_bounds=lhs_parameters.get("lower_bounds"),
    upper_bounds=lhs_parameters.get("upper_bounds"),
    num_samples=lhs_parameters.get("num_samples"),
)

# Add Hs and Hs_Lo to the lhs dataset
lhs_dataset["Hs"] = Hs
lhs_dataset["Hs_L0"] = Hs_Lo
# lhs_dataset["hv"]=1
# lhs_dataset["Nv"]=1000
# Predict the PCA components using the RBF model

Ru_dataset=rbf_Ru.predict(lhs_dataset)

In [51]:
subset_data=mda.centroids.iloc[postprocessed_output["case_num"].values, :]

#postprocessed_output["Ru2"].values
subset_data["ru2"]=postprocessed_output["Ru2"].values
subset_data

#subset_data['Ru2']=postprocessed_output["Ru2"].values

/tmp/ipykernel_2482747/4105333724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data["ru2"]=postprocessed_output["Ru2"].values


,Hs,Hs_L0,Wv,hv,Nv,ru2
0,2.868944,0.009762,192.969099,1.305966,905.293031,0.193260
1,0.534095,0.004325,3.818998,0.082704,17.239093,0.194340
2,0.700457,0.003186,194.041983,0.545457,941.743602,0.204248
3,2.904176,0.003385,32.668842,1.498247,248.974443,0.207336
4,1.671713,0.009469,198.962239,0.000809,42.061439,0.209128
...,...,...,...,...,...,...
995,1.338252,0.007070,154.734375,0.736468,23.686795,2.164960
996,2.726273,0.008474,89.976963,0.246053,398.930722,2.193120
997,2.246973,0.003430,38.894942,0.771885,996.961851,2.193320
998,2.081535,0.009883,3.815951,0.287924,263.982976,2.222300


In [40]:
lhs_dataset["Ru2"]=Ru_dataset.Ru2
lhs_dataset.sort_values(by=["Ru2"], ascending=False, inplace=True)

# Select lhs_dataset with Wv < 100
#lhs_dataset = lhs_dataset[lhs_dataset["Wv"] < 100].copy(deep=True)
lhs_dataset


,Wv,hv,Nv,Hs,Hs_L0,Ru2
366,199.618800,1.439663,999.658578,2.5,0.005,1.027537
155,197.714333,1.403807,999.132474,2.5,0.005,1.024971
236,199.466507,1.465336,999.830413,2.5,0.005,1.023183
228,198.627722,1.456544,999.122696,2.5,0.005,1.019154
361,198.905437,1.471263,999.236366,2.5,0.005,1.019055
...,...,...,...,...,...,...
235,0.385631,1.427791,999.540263,2.5,0.005,-0.047237
286,2.624672,1.477791,999.113511,2.5,0.005,-0.051585
288,1.811521,1.493062,999.799322,2.5,0.005,-0.066119
127,1.222042,1.485438,999.754041,2.5,0.005,-0.067456
